### Libraries Needed

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import datetime as dt
from scipy.optimize import brentq


# Extract local volatility

Extracting local volatility is crucial for modeling the dynamics of financial assets. Local volatility models allow the asset's volatility to vary with both price and time, providing a more accurate representation than constant volatility models.

To begin, we'll need to gather market data, including historical prices of options, spot prices, risk-free interest rates, and dividends. Using this data, we can construct the implied volatility surface, which displays implied volatility for various strike prices and maturities.

In [13]:
def option_chains(ticker):
    """
    """
    asset = yf.Ticker(ticker)
    expirations = asset.options
    chains = pd.DataFrame()
    for expiration in expirations:
        opt = asset.option_chain(expiration)
        calls = opt.calls
        calls['optionType'] = "call"
        puts = opt.puts
        puts['optionType'] = "put"
        chain = pd.concat([calls, puts])
        chain['expiration'] = pd.to_datetime(expiration) + pd.DateOffset(hours=23, minutes=59, seconds=59)
        chains = pd.concat([chains, chain])
    chains["daysToExpiration"] = (chains.expiration - dt.datetime.today()).dt.days + 1
    return chains

In [17]:
options = option_chains("AAPL")
calls = options[options["optionType"] == "call"]
put = options[options["optionType"] == "put"]

I noticed that on Yahoo Finance, implied volatilities are provided, but the specific method used to compute them isn't clear to me. To gain a deeper understanding of how implied volatilities are constructed, I've decided to compute them myself. This approach will allow me to delve into well-known financial mathematical formulas and better grasp their application.

To compute implied volatilities accurately, one typically needs to employ numerical methods. These methods aim to find the volatility value that, when input into the Black-Scholes formula, results in the market price of the option. A commonly used approach involves employing root-finding algorithms such as the Newton-Raphson method or Brent's method. I choose to calculate the two root-finding algorithms and compare them with the implied volatilities from Yahoo Finance. 

The Black-Scholes formula is foundational in options pricing, serving as a starting point for more sophisticated models that accommodate additional factors such as dividends, early exercise features, and varying interest rates. Its insights into risk-neutral valuation have significantly shaped modern finance.

The Black-Scholes formula Call and Put formula are :
$$
{\boxed{C(S_{0},K,r,T,\sigma )=S_{0}{\mathcal {N}}(d_{1})-K\mathrm {e} ^{-rT}{\mathcal {N}}(d_{2})}}
$$

$$
{\boxed{P(S_{0},K,r,T,\sigma )=-S_{0}{\mathcal {N}}(-d_{1})+K\mathrm {e} ^{-rT}{\mathcal {N}}(-d_{2})}}
$$

with : 
- $\mathcal {N}$ :  la fonction de répartition de la loi normale centrée réduite $\mathcal {N}{(0,1)}$
- $d1 = {\frac {1}{\sigma {\sqrt {T}}}}\left[\ln \left({\frac {S_{0}}{K}}\right)+\left(r+{\frac {1}{2}}\sigma ^{2}\right)T\right]$  
- $d2 = d1-\sigma {\sqrt {T}}$
- $S_0$ the current underlying price
- $K the strike option price
- $r$ the risk-free interest rate
- $T$ the time to maturity of the option
- $\sigma$ the volatility of the underlying asset


In [29]:
def bs_call_price(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
def bs_put_price(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return -S * norm.cdf(*d1) + K * np.exp(-r * T) * norm.cdf(-d2)

The Newton-Raphson method (also known as Newton's method) is an iterative root-finding algorithm that uses the derivative of a function to iteratively refine an initial guess for the root. It converges quickly for well-behaved functions but requires the derivative to be continuous and non-zero near the root.

The iterative formula for Newton-Raphson method is given by:

$x_{n+1} = x_{n} - \frac{f(x_{n})}{f'(x_{n})}$

where:
- $x_{n}$ is the current estimate of the root 
- $f'(x_{n})$ is the derivative of $f(x)$ with respect to $ x $.

In [23]:
def newton_raphson(f, df, x0, tol=1e-6, max_iter=100):
    """
    Newton-Raphson method for finding the root of f(x) = 0.

    Parameters:
    f : function
        The function for which to find the root.
    df : function
        The derivative of f(x).
    x0 : float
        Initial guess for the root.
    tol : float, optional
        Tolerance (stopping criterion). Default is 1e-6.
    max_iter : int, optional
        Maximum number of iterations. Default is 100.

    Returns:
    float
        Approximation of the root.
    """
    x = x0
    for i in range(max_iter):
        fx = f(x)
        if abs(fx) < tol:
            return x
        dfx = df(x)
        if dfx == 0:
            break  
        x = x - fx / dfx
    return x


Brent's method is another root-finding algorithm that combines the robustness of the bisection method with the speed of methods such as secant method, bisection method and inverse quadratic interpolation. It does not require the derivative of the function and generally converges faster than bisection.

Brent's method iteratively narrows down the interval containing the root by:
- <ins>Bisection method</ins> : Given an interval $[a, b]$ where $ f(a) $ and $ f(b) $ have opposite signs (i.e., $ f(a) \cdot f(b) < 0 $), the iterative formula for the bisection method is: $ \boxed{c = \frac{a + b}{2} }$
    - If $ f(c) = 0 $, then $ c $ is the root.
    - If $ f(a) \cdot f(c) < 0 $, then the root lies in $[a, c]$.
    - If $ f(b) \cdot f(c) < 0 $, then the root lies in $[c, b]$.

- <ins>Secant method</ins> : Given two initial guesses $ x_0 $ and $ x_1 $, the iterative formula for the secant method is: $\boxed{ x_{n+1} = x_n - \frac{f(x_n) \cdot (x_n - x_{n-1})}{f(x_n) - f(x_{n-1})} }$
    - $ x_n $ is the current estimate of the root,
    - $ f(x_n) $ and $ f(x_{n-1}) $ are the values of the function at $ x_n $ and $ x_{n-1} $, respectively.

- <ins>Inverse quadratic interpolation</ins> : Given three points $ x_0, x_1, x_2 $ where $ f(x_0), f(x_1), f(x_2) \neq 0 $, the formula for the new approximation $ x_3 $ is:

     $\boxed{ x_3 = x_0 - \frac{2 \left[ (x_0 - x_1)(f(x_0) - f(x_2)) - (x_0 - x_2)(f(x_0) - f(x_1)) \right]}{(x_0 - x_1)^2 (f(x_0) - f(x_2)) - (x_0 - x_2)^2 (f(x_0) - f(x_1))}}$

    - $ x_3 $ is the new approximation of the root,
    - $ x_0, x_1, x_2 $ are three successive points used for interpolation,
    - $ f(x_0), f(x_1), f(x_2) $ are the values of the function evaluated at $ x_0, x_1, x_2 $, respectively.



In [26]:
def objective_function(x):
    return bs_call_price(S, K, T, r, sigma) - market_price

Now that we got the numerical methods and the objective function (BS result - Yahoo Finance prices), we can compute these formula to have implied volatilities.

In [27]:
calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,optionType,expiration,daysToExpiration
0,AAPL240705C00100000,2024-06-28 18:43:31+00:00,100.0,113.50,120.45,122.40,0.000000,0.000000,7.0,0,5.882815,True,REGULAR,USD,call,2024-07-05 23:59:59,2
1,AAPL240705C00105000,2024-06-27 15:04:34+00:00,105.0,108.42,115.40,117.50,0.000000,0.000000,NaN,1,5.673831,True,REGULAR,USD,call,2024-07-05 23:59:59,2
2,AAPL240705C00110000,2024-07-01 17:18:48+00:00,110.0,107.35,110.45,112.40,0.000000,0.000000,1.0,0,5.250003,True,REGULAR,USD,call,2024-07-05 23:59:59,2
3,AAPL240705C00125000,2024-06-28 18:44:51+00:00,125.0,88.39,95.40,97.50,0.000000,0.000000,2.0,0,4.491215,True,REGULAR,USD,call,2024-07-05 23:59:59,2
4,AAPL240705C00130000,2024-06-28 17:52:13+00:00,130.0,84.01,90.50,92.40,0.000000,0.000000,1.0,1,4.134770,True,REGULAR,USD,call,2024-07-05 23:59:59,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,AAPL261218C00310000,2024-07-02 19:36:10+00:00,310.0,14.55,14.75,15.75,0.000000,0.000000,2.0,92,0.290183,False,REGULAR,USD,call,2026-12-18 23:59:59,898
39,AAPL261218C00320000,2024-07-03 16:52:57+00:00,320.0,13.35,13.05,13.80,0.600000,4.705885,1.0,328,0.287071,False,REGULAR,USD,call,2026-12-18 23:59:59,898
40,AAPL261218C00330000,2024-07-03 16:26:58+00:00,330.0,11.93,11.60,12.20,0.380000,3.290044,7.0,471,0.285377,False,REGULAR,USD,call,2026-12-18 23:59:59,898
41,AAPL261218C00340000,2024-07-03 16:40:06+00:00,340.0,10.52,10.20,10.80,0.400001,3.952575,110.0,0,0.284034,False,REGULAR,USD,call,2026-12-18 23:59:59,898


In [ ]:
implied_vols = []
for K, price in zip(strikes, market_prices):
    initial_guess = 0.2
    implied_vol = brentq(implied_vol_objective, 1e-6, 1.0, args=(price, S, K, T, r), xtol=1e-6, rtol=1e-6, maxiter=100)
    implied_vols.append(implied_vol)